# Multihop Task Execution with OpenAPI

Using robust OpenAPI Chain primitives, LLM Chains can efficiently plan and combine calls across multiple endpoints. This notebook demonstrates a sample composition of Speak and Klarna APIs.

For a detailed walkthrough, see the [OpenAPI Operation Chain](openapi.ipynb) notebook.

### First, import dependencies and load the LLM

In [1]:
from typing import List, Optional
from langchain.chains import LLMChain, OpenAPIEndpointChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.requests import Requests
from langchain.tools import APIOperation, OpenAPISpec
from langchain.agents import AgentType, Tool, initialize_agent

In [2]:
# Select the LLM to use. Here, we use text-davinci-003
llm = OpenAI(temperature=0, max_tokens=700) # You can swap between different core LLM's here.

### Next, load the OpenAPI specification

In [3]:
speak_spec = OpenAPISpec.from_url("https://api.speak.com/openapi.yaml")
klarna_spec = OpenAPISpec.from_url("https://www.klarna.com/us/shopping/public/openai/v0/api-docs/")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


### Create the OpenAPIEndpoint Tools

Create an OpenAPIEndpointChain for each of the API's HTTP methods. For larger OpenAPI specs, you may find it easier to select a subset of the operations that work well together. Then we can package these chains up as tools to pass to a conversational agent.

In [4]:
def chain_to_tool(chain: OpenAPIEndpointChain, domain_title: str) -> Tool:
    """Convert an endpoint chain to a tool."""
    expanded_name = f'{domain_title.replace(" ", "_")}.{chain.api_operation.operation_id}'
    return Tool(
        name=expanded_name,
        func=chain,
        description=chain.api_operation.description
    )

# Headers can be passed in for user authentication to the endpoint.
def make_endpoint_tools(spec: OpenAPISpec, headers: Optional[dict] = None) -> List[Tool]:
    """Create a chain for each endpoint in the OpenAPI spec."""
    http_operation_tools = []
    for path in spec.paths:
        for method in spec.get_methods_for_path(path):
            api_operation = APIOperation.from_openapi_spec(spec, path, method)
            chain = OpenAPIEndpointChain.from_api_operation(
                api_operation, 
                llm, 
                requests=Requests(headers=headers), 
                verbose=False,
                return_intermediate_steps=False,
                max_text_length=2000 # Truncate large request responses
            )
            endpoint_tool = chain_to_tool(chain, spec.info.title)
            http_operation_tools.append(endpoint_tool)
    return http_operation_tools

In [5]:
klarna_tools = make_endpoint_tools(klarna_spec)
speak_tools = make_endpoint_tools(speak_spec)
open_api_tools = klarna_tools + speak_tools

### Create the Agent

In [6]:
# Slightly tweak the action input from the default instructions to pass more context to the endpoints
openapi_format_instructions = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: fully specify what the action should do or answer
Observation: The tool's reply
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

In [7]:
mrkl = initialize_agent(open_api_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                        verbose=True, format_instructions=openapi_format_instructions)

In [8]:
mrkl.run("I have an end of year party for my Italian class and have to buy some Italian clothes for it")



> Entering new AgentExecutor chain...
 I need to find out what Italian clothes are available
Action: Open_AI_Klarna_product_Api.productsUsingGET
Action Input: Italian clothes
Observation: {'instructions': 'Italian clothes', 'output': 'We have two Italian-style clothing items available: Alé Colour Block Short Sleeve Jersey Men - Italian Blue and Alé Dolid Flash Jersey Men - Italian Blue. Both are available for purchase on our website.'}
Thought: I now know what Italian clothes are available
Final Answer: We have two Italian-style clothing items available: Alé Colour Block Short Sleeve Jersey Men - Italian Blue and Alé Dolid Flash Jersey Men - Italian Blue. Both are available for purchase on our website.

> Finished chain.


'We have two Italian-style clothing items available: Alé Colour Block Short Sleeve Jersey Men - Italian Blue and Alé Dolid Flash Jersey Men - Italian Blue. Both are available for purchase on our website.'

### Adding more Endpoints

Since each OpenAPIEndpoint tool exposes a natural language interface, the conversational agent has an easier job planning each step. Let's add another API to the chain.

Adding the Spoonacular endpoints.

1. Go to the [Spoonacular API Console](https://spoonacular.com/food-api/console#Profile) and make a free account.
2. Click on `Profile` and copy your API key below.

In [9]:
spoonacular_spec = OpenAPISpec.from_url("https://spoonacular.com/application/frontend/downloads/spoonacular-openapi-3.json")
spoonacular_api_key = "" # Copy from the API Console

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [10]:
spoonacular_tools = make_endpoint_tools(spoonacular_spec, headers={"x-api-key": spoonacular_api_key})
open_api_tools = klarna_tools + speak_tools + spoonacular_tools
print(f"{len(open_api_tools)} tools loaded.")

Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'quer

103 tools loaded.


In [11]:
# Make the query more complex!
user_input = (
    "I'm learning Italian, and my language class is having an end of year party... "
    " Could you help me find an Italian outfit to wear and"
    " an appropriate recipe to prepare so I can present for the class in Italian?"
)

In [12]:
mrkl.run(user_input)



> Entering new AgentExecutor chain...
 I need to find a product and a recipe in Italian.
Action: Open_AI_Klarna_product_Api.productsUsingGET
Action Input: Italian outfit
Observation: {'instructions': 'Italian outfit', 'output': 'We have several Italian-style products available, including the Italian Gold Sparkle Perfectina Necklace - Gold, Italian Gold Claddagh Ring - Gold, and Italian Gold Herringbone Necklace - Gold. Please click on the links to view more information about each product.'}
Thought: I need to find a recipe in Italian.
Action: Speak.translate
Action Input: Italian recipe
Observation: {'instructions': 'Italian recipe', 'output': "You can try making an Italian dish using a recipe for Italian cuisine, or you can try cooking Italian dishes without a recipe. You can also say 'Cuciniamo all'italiana!' which means 'Let's cook like the Italians do!'"}
Thought: I now know the final answer.
Final Answer: You can find an Italian outfit to wear by using Open_AI_Klarna_product_Api

'You can find an Italian outfit to wear by using Open_AI_Klarna_product_Api.productsUsingGET and you can find an appropriate recipe to prepare by using Speak.translate.'

## Thank you!

In [13]:
open_api_tools[1].run("Tell the LangChain audience to 'enjoy the meal' in Italian, please!")['output']

'Mangia bene!'